<a href="https://colab.research.google.com/github/AmtulYasmeen/Projects-Done/blob/main/AV_jobathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install catboost

     |████████████████████████████████| 65.7MB 63kB/s 


In [2]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier,StackingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer, RobustScaler
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold,train_test_split
import gc
import datetime
from tqdm.notebook import tqdm ,tnrange
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_columns', 500)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
# loading train data
df_train =pd.read_csv('/content/gdrive/My Drive/AV jobathon/train_AV.csv')
# loading test data
df_test =pd.read_csv('/content/gdrive/My Drive/AV jobathon/test_AV.csv')
df_sub = pd.read_csv('/content/gdrive/My Drive/AV jobathon/sub_df.csv')

In [13]:
def clean_data(df_train, df_test):
  df_train = df_train.drop_duplicates()
  df_train = df_train[~df_train.Region_Code.isin(list(set(df_train.Region_Code)-set(df_test.Region_Code)))]
  df_all = pd.concat([df_train,df_test],ignore_index=True, sort=False)
  print("Shape of datasets: ", df_train.shape, df_test.shape, df_all.shape)
  df_all['holds_insurance_policy'] = np.where(df_all['Holding_Policy_Duration'].isnull(), 0,1)
  df_all['Holding_Policy_Duration'].fillna(0,inplace=True) 
  df_all['Holding_Policy_Duration'].replace('14+',15,inplace=True) 
  df_all['Holding_Policy_Duration'] = df_all['Holding_Policy_Duration'].astype(float).astype(int)
  df_all['Holding_Policy_Type'].fillna(0,inplace=True)
  df_all['Health Indicator'].fillna('X0',inplace=True)
  df_all['Age_difference'] = df_all['Upper_Age'] - df_all['Lower_Age']
  df_all['Average_age'] = (df_all['Upper_Age'] + df_all['Lower_Age'])/2
  return df_all
  


In [14]:
df_all = clean_data(df_train, df_test)

Shape of datasets:  (48752, 14) (21805, 13) (70557, 14)


In [15]:
# combining 2 features and to label encode them later
for col in ['Health Indicator','Holding_Policy_Duration','Holding_Policy_Type',
            'Reco_Insurance_Type','Reco_Policy_Cat','holds_insurance_policy',
           'Accomodation_Type','Is_Spouse'
           ]:
    df_all['City_'+str(col)] = df_all['City_Code'].astype(str)+'_'+df_all[col].astype(str) 
    df_all['Region_'+str(col)] = df_all['Region_Code'].astype(str)+'_'+df_all[col].astype(str)
    if col != 'Reco_Insurance_Type':
      df_all['Reco_Insurance_Type_'+ str(col)] = df_all['Reco_Insurance_Type'].astype(str) + '_' + df_all[col].astype(str)
    if col != 'Reco_Policy_Cat':
      df_all['Reco_Policy_Cat_' + str(col)] = df_all['Reco_Policy_Cat'].astype(str) + '_' + df_all[col].astype(str)
df_all['Holding_Policy_Duration_Type'] = df_all['Holding_Policy_Duration'].astype(str)+'_'+df_all['Holding_Policy_Type'].astype(str)

df_all['Reco_Insurance_Type_Cat'] = df_all['Reco_Insurance_Type'].astype(str)+'_'+df_all['Reco_Policy_Cat'].astype(str)

In [16]:
# to_be_added_df = pd.DataFrame()
# to_be_added_df['Accomodation_Type'] = df_all['Accomodation_Type']
# to_be_added_df['Reco_Insurance_Type'] = df_all['Reco_Insurance_Type']
# to_be_added_df['Is_Spouse'] = df_all['Is_Spouse']

In [17]:
# removing anything other than number from city_code and health indicator and converting it into int type
df_all['City_Code'] = df_all['City_Code'].str.replace("[^0-9]","")
df_all['City_Code'] = df_all['City_Code'].astype(float).astype(int)

df_all['Health Indicator'] = df_all['Health Indicator'].str.replace("[^0-9]","")
df_all['Health Indicator'] = df_all['Health Indicator'].astype(float).astype(int)

# creating dummy variable based on value_count of the variables
df_all = pd.get_dummies(df_all,columns=['Accomodation_Type','Reco_Insurance_Type','Is_Spouse'])


In [18]:
# label encoding the categorical variable
le = LabelEncoder()
for col in df_all.columns:
    if (df_all[col].dtypes == "object"):
#         df_all[col].fillna("not_available", inplace = True)
        df_all[col] = df_all[col].astype(str)
        df_all[col] = le.fit_transform(df_all[col])
#         df_all[col] = df_all[col].astype('category')

In [19]:
# transforming continous variables into 8 to 10 buckets 
num_feature_list = ['Reco_Policy_Premium', 'Upper_Age' ,'Lower_Age' ,'Age_difference', 'Average_age']

for feature in num_feature_list:
  if feature != 'Reco_Policy_Premium':
    age_discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
    df_all[feature + '_Bins'] = age_discretizer.fit_transform(df_all[feature].values.reshape(-1,1)).astype(int)
  else:
    premium_discretizer = KBinsDiscretizer(n_bins=8, encode='ordinal', strategy='quantile')
    df_all[feature + '_Bins'] = premium_discretizer.fit_transform(df_all[feature].values.reshape(-1,1)).astype(int)


In [20]:
 for col in ['City_Code','Region_Code','Upper_Age','Lower_Age','Health Indicator','Holding_Policy_Duration','Holding_Policy_Type',
           'Reco_Policy_Cat','holds_insurance_policy']:
    
    val_counts = df_all[col].value_counts().to_dict()
    df_all[str(col)+"_counts"] = df_all[col].map(val_counts)

In [21]:
def frequency_encoding(column_name,output_column_name,df):
    fe_pol = (df.groupby(column_name).size()) / len(df)
    df[output_column_name] = df[column_name].apply(lambda x : fe_pol[x])

In [22]:
def agg_1_feature(df, col):
  col_dict = {'Lower_Age': ['mean', 'max', 'min','std'],
              'holds_insurance_policy': ['mean', 'max', 'min','std','sum'],
              'Holding_Policy_Duration': ['mean', 'max', 'min','std','sum'],
              'Average_age': ['mean', 'max', 'min','std','sum'],
              'Reco_Policy_Premium': ['mean', 'max', 'min','std','sum'], 
              'City_Code' : ['nunique','count'],
              'Region_Code': ['nunique','count'], 
              'Health Indicator': ['nunique','count'] ,
              'Holding_Policy_Type': ['nunique','count'] ,
              'Reco_Policy_Cat': ['nunique','count']}
  rem_list = [col]
  for key in rem_list:
    col_dict.pop(key, None) 
  
  col_name = df.groupby([col]).agg(col_dict)
  col_name.columns = [f'{col}_' + '_'.join(c) for c in col_name.columns]
  df = pd.merge(df, col_name, on = [col], how='left')
  return col_name.columns

In [23]:
def agg_2_feature(df, col1, col2): 
  col_dict = {'Upper_Age': ['mean', 'max', 'min','std'],
              'Lower_Age': ['mean', 'max', 'min','std'],
              'holds_insurance_policy': ['mean', 'max', 'min','std','sum'],
              'Holding_Policy_Duration': ['mean', 'max', 'min','std','sum'],
              'Average_age': ['mean', 'max', 'min','std','sum'],
              'Reco_Policy_Premium': ['mean', 'max', 'min','std','sum'], 
              'City_Code' : ['nunique','count'],
              'Region_Code': ['nunique','count'],
              'Health Indicator': ['nunique','count'] ,
              'Holding_Policy_Type': ['nunique','count'] ,
              'Reco_Policy_Cat': ['nunique','count']}
  rem_list = [col1,col2]
  for key in rem_list:
    col_dict.pop(key, None) 
  # col_name = f'{col1}_{col2}_aggregate_features'
  col_name = df.groupby([col1, col2]).agg(col_dict)
  col_name.columns = [f'{col1}_{col2}_' + '_'.join(c) for c in col_name.columns]
  df = pd.merge(df, col_name, on = [col1, col2], how='left')
  return col_name.columns


In [24]:
def num_feature_agg(df,grpby_feature, num_feature ):
      featured = df.groupby([grpby_feature]).agg({ num_feature : ['mean', 'max', 'min', 'std']})
      featured.columns = [f'{grpby_feature}_{num_feature}' + '_'.join(c).strip('_') for c in featured.columns]
      df = pd.merge(df, featured, on = [grpby_feature], how='left')

In [25]:
# agg_1_feature(df,col)  agg_2_feature(df, col1, col2) num_feature_agg(df, grpby_feature, num_feature)

In [26]:
from itertools import combinations

In [27]:
def feature_engineering(df):
    cat_features=[]
    
    #Interaction Feature (Combining 2 categorical features and performing frequency encoding)
    columns=['City_Code','Health Indicator'
            ,'Region_Code','Holding_Policy_Type','Reco_Policy_Cat']
  
    comb = combinations(columns, 2) 

    for i in list(comb):  
        df[f'{i[0]}_{i[1]}']=df[i[0]].astype(str)+'_'+df[i[1]].astype(str)
        frequency_encoding(f'{i[0]}_{i[1]}',f'{i[0]}_{i[1]}',df)
        cat_features.append(f'{i[0]}_{i[1]}')
        frequency_encoding('City_Code','City_Code_fe',df)
        frequency_encoding('Region_Code','Region_Code_fe',df)
        frequency_encoding('Holding_Policy_Duration','Holding_Policy_Duration',df)
        frequency_encoding('Holding_Policy_Type','Holding_Policy_Type_fe',df)
        #Deriving characteristics of each city by creating aggregate features

    for i in cat_features:
      for j in ['max', 'min','mean','std']:
        df[f'city_{i}_{j}']=df.groupby('City_Code')[i].transform(j)
        df[f'city_region_{i}_{j}']=df.groupby(['City_Code','Region_Code'])[i].transform(j)
        df[f'city_reco_policycat_{i}_{j}']=df.groupby(['City_Code','Reco_Policy_Cat'])[i].transform(j)

    #creating aggregate features
    col1_list = ['City_Code','Lower_Age','Lower_Age_Bins', 'Holding_Policy_Type', 'Health Indicator', 'Region_Code_Reco_Policy_Cat','City_Code_Region_Code',
     'Holding_Policy_Type_Reco_Policy_Cat', 'City_Code_Reco_Policy_Cat']
    for col in col1_list:
      agg_1_feature(df,col)

    col2_list = [('City_Code','Region_Code'), ('City_Code','Reco_Policy_Cat'), ('City_Code','Region_Code_Reco_Policy_Cat')]
    for i, j in col2_list:
      agg_2_feature(df, f'{i}', f'{j}')

    for i in ['Holding_Policy_Type_Reco_Policy_Cat', 'City_Code_Reco_Policy_Cat', 'Region_Code_Reco_Policy_Cat']:
      num_feature_agg(df, f'{i}', 'Reco_Policy_Premium')

    return df,cat_features
    


    

In [28]:
df_all, cat_features = feature_engineering(df_all)

In [29]:
df_all.shape

(70557, 199)

In [30]:
train_x_full = df_all[df_all['Response'].notnull()]

cols = ['City_Code', 'Region_Code',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium']

train_x_full = train_x_full[~train_x_full.loc[:,:].duplicated(keep = 'first')].reset_index(drop=True)

test_x_full = df_all[df_all['Response'].isnull()]
train_y_full = train_x_full['Response']
test_id=test_x_full['ID']
del test_x_full['Response']
del train_x_full['Response']
del test_x_full['ID']
del train_x_full['ID']
print(train_x_full.shape, train_y_full.shape, test_x_full.shape)

(48752, 197) (48752,) (21805, 197)


In [31]:
for col in ['City_Code', 'Region_Code', 
               'Health Indicator', 'Holding_Policy_Type',
               'Reco_Policy_Cat','holds_insurance_policy', 
               'City_Health Indicator', 'Region_Health Indicator',
               'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
               'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
               'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
               'City_holds_insurance_policy', 'Region_holds_insurance_policy',
               'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
               'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
               'Reco_Insurance_Type_Cat', 'Accomodation_Type_Rented',
               'Reco_Insurance_Type_Joint', 'Is_Spouse_Yes',
               'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
               'Holding_Policy_Duration','Reco_Policy_Premium',
               'Upper_Age', 'Lower_Age','Age_difference',
               'Accomodation_Type_Rented','Reco_Insurance_Type_Joint', 'Is_Spouse_Yes'
              ]:
    train_x_full[col] = train_x_full[col].astype(int)
    test_x_full[col] = test_x_full[col].astype(int)
cat_features = ['City_Code', 'Region_Code', 
               'Health Indicator', 'Holding_Policy_Type',
               'Reco_Policy_Cat','holds_insurance_policy', 
               'City_Health Indicator', 'Region_Health Indicator',
               'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
               'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
               'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
               'City_holds_insurance_policy', 'Region_holds_insurance_policy',
               'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
               'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
               'Reco_Insurance_Type_Cat', 'Accomodation_Type_Rented',
               'Reco_Insurance_Type_Joint', 'Is_Spouse_Yes',
               'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
               'Holding_Policy_Duration','Reco_Policy_Premium',
               'Accomodation_Type_Rented','Reco_Insurance_Type_Joint', 'Is_Spouse_Yes'
              ]

In [32]:
train_pred_all = pd.DataFrame()

In [33]:
# training
train_x = train_x_full.copy()
train_y = train_y_full.copy()
test_x = test_x_full.copy()
feature_importance_df = pd.DataFrame()

NFOLDS = 2
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=18121995)
pred_test = []
pred_x = []
pred_y = []
j=1
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('***Fold*** :', fold+1)
    model = CatBoostClassifier(n_estimators=20000,random_state=1812195,learning_rate=0.03,eval_metric='AUC',
                              cat_features =cat_features)
    model.fit(train_x.loc[train_ids], train_y.loc[train_ids], 
              eval_set=[(train_x.loc[train_ids], train_y.loc[train_ids]), (train_x.loc[test_ids], train_y.loc[test_ids])],
              verbose=500,
              early_stopping_rounds=200)
    pred_fold = model.predict_proba(train_x.loc[test_ids])[:,-1]
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_fold_test = model.predict_proba(test_x)[:,-1]
    pred_test.append([float(i) for i  in pred_fold_test])
    print('\n')
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

print("Final ROC-AUC Score:", roc_auc_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

vote = final_prediction.mean(axis=1)
final_prediction['Response'] = vote
final_prediction['ID'] = test_id.values
print(final_prediction.shape)

train_pred_all['cb_pred'] = pred_x
final_prediction_cb = final_prediction.copy()

final_prediction.head()

***Fold*** : 1
0:	test: 0.7371368	test1: 0.7293266	best: 0.7293266 (0)	total: 471ms	remaining: 2h 37m 9s
500:	test: 0.9016572	test1: 0.8088011	best: 0.8088011 (500)	total: 2m 47s	remaining: 1h 48m 49s
1000:	test: 0.9146768	test1: 0.8098326	best: 0.8100812 (832)	total: 5m 44s	remaining: 1h 49m 3s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8100812332
bestIteration = 832

Shrink model to first 833 iterations.


***Fold*** : 2
0:	test: 0.7566583	test1: 0.7556131	best: 0.7556131 (0)	total: 426ms	remaining: 2h 21m 54s
500:	test: 0.8735780	test1: 0.8116399	best: 0.8116399 (500)	total: 2m 50s	remaining: 1h 50m 47s
1000:	test: 0.8882574	test1: 0.8125767	best: 0.8125938 (970)	total: 5m 38s	remaining: 1h 47m 12s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8126081754
bestIteration = 1081

Shrink model to first 1082 iterations.


Final ROC-AUC Score: 0.8112137101322873
(21805, 4)


,FOLD_0,FOLD_1,Response,ID
0,0.087655,0.041801,0.064728,50883
1,0.076361,0.098373,0.087367,50884
2,0.449231,0.571707,0.510469,50885
3,0.091489,0.108810,0.100150,50886
4,0.039401,0.021286,0.030344,50887


In [34]:
train_pred_all['cb_pred'] = pred_x
final_prediction_cb = final_prediction.copy()

final_prediction.head()

,FOLD_0,FOLD_1,Response,ID
0,0.087655,0.041801,0.064728,50883
1,0.076361,0.098373,0.087367,50884
2,0.449231,0.571707,0.510469,50885
3,0.091489,0.108810,0.100150,50886
4,0.039401,0.021286,0.030344,50887


In [35]:

df_submission = final_prediction[['ID', 'Response']]
df_submission.to_csv("submission.csv", index = False)
df_submission.head()

,ID,Response
0,50883,0.064728
1,50884,0.087367
2,50885,0.510469
3,50886,0.100150
4,50887,0.030344


In [36]:
df_submission.tail(10)

,ID,Response
21795,72678,0.089273
21796,72679,0.658521
21797,72680,0.480276
21798,72681,0.127538
21799,72682,0.020111
21800,72683,0.043738
21801,72684,0.067546
21802,72685,0.027322
21803,72686,0.561753
21804,72687,0.150558


NOTE: referenced from anil betta and krishna priya notebooks